<a href="https://colab.research.google.com/github/Josh-Been/SBL-Freedom-Deuteronomy/blob/master/FuzzyTextMatching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[PPTX](https://baylor.box.com/s/v31v722ueyvag6l8hnvi7nii02pes57z)

Specify Source File
### Must be a .TXT text file

In [0]:
from google.colab import files
from google.colab import output

# Browse/Upload File
up=files.upload()
# File passed to variable doc
sourcedoc=next(iter(up))

try:
    f=open(sourcedoc,'r', encoding='utf-8-sig')
    source=f.read()
    f.close()
except:
    f=open(sourcedoc,'r', encoding='ISO-8859-1')
    source=f.read()
    f.close()

Specify Target File
### Must be a .TXT text file

In [0]:
# Browse/Upload File
up=files.upload()
# File passed to variable doc
targetdoc=next(iter(up))

try:
    f=open(targetdoc,'r', encoding='utf-8-sig')
    target=f.read()
    f.close()
except:
    f=open(targetdoc,'r', encoding='ISO-8859-1')
    target=f.read()
    f.close()

Specify:


1.   Ngram Size
2.   Fuzzy Method
3.   Minimuim Match Score
4.   Speed vs. Accuracy



In [0]:
#################################
ngram_size=10
fuzzy_method='ratio'  #[ratio, partial ratio, token sort ratio, token set ratio]
minimum_score=80
speed=3    # 1 is slowest, represents the nth ngrams checked in the target file. 1 is every ngram. 2 is every other ngram. 3 is every third ngram.
#################################

!pip install python-Levenshtein
try:
    from fuzzywuzzy import fuzz
except:
    !pip install fuzzywuzzy
    from fuzzywuzzy import fuzz
output.clear()

import csv, re, math

def strip_non_ascii(cleanme):
    cleanme = re.sub(r'[^\w\s]','',cleanme)
    cleanme = cleanme.replace('\n',' ')
    cleanme = cleanme.replace('\r',' ')
    cleanme = cleanme.lower()
    cleanme = ' '.join([word for word in cleanme.split() if not 'page' in word and not word.isdigit()])
    return ''.join([i if ord(i) < 128 else ' ' for i in cleanme])

def ngrams(input, n):
    input = input.split(' ')
    output = []
    for i in range(len(input)-n+1):
        output.append(' '.join(input[i:i+n]))
    return output

def roundup(x):
    return int(math.ceil(x / 100.0)) * 100

source_ngrams=(ngrams(strip_non_ascii(source),ngram_size))
target_ngrams=(ngrams(strip_non_ascii(target),ngram_size))

matches=[sourcedoc.replace(',','')+','+targetdoc.replace(',','')+',Similarity\n']
i=0
skip=0

for s in source_ngrams:
    skip=skip-1
    i+=1
    if i==1 or i%100==0:
        print('Checking Ngram '+str(i)+'-'+str(roundup(i+99))+'/'+str(len(source_ngrams)))
    if skip <=0:
        for target_skip in range(0,len(target_ngrams)):
            if target_skip==0 or target_skip%speed==0:
                if fuzzy_method=='ratio':
                    measure=fuzz.ratio(s,target_ngrams[target_skip])
                elif fuzzy_method=='partial ratio':
                    measure=fuzz.partial_ratio(s,target_ngrams[target_skip])
                elif fuzzy_method=='token sort ratio':
                    measure=fuzz.token_sort_ratio(s,target_ngrams[target_skip])
                elif fuzzy_method=='token set ratio':
                    measure=fuzz.token_set_ratio(s,target_ngrams[target_skip])
                if measure>=minimum_score:
                    matches.append(s+','+target_ngrams[target_skip]+','+str(measure)+'\n')
                    print(measure, '  >>  ', s, '|', target_ngrams[target_skip])
                    skip=ngram_size/2

f=open('similarities.csv','w', encoding='utf-8-sig')
for match in matches:
    f.write(match)
f.close()

print('\nCompleted!')